In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
from dask.distributed import Client
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True)

import ipywidgets as widgets
from IPython.display import display

In [35]:
ds = xr.load_dataset("../data/outputs/plots/metrics/metrics_grid_none_z_1m.nc")

# Site level height profiles

In [3]:
sites = np.unique(ds.site.values)

In [36]:
# Explore dataset structure to understand site types
print("Dataset dimensions and coordinates:")
print(ds.dims)
print("\nDataset coordinates:")
print(ds.coords)
print("\nDataset data variables:")
print(ds.data_vars)
print("\nSite values:")
print(ds.site.values)

# Check if there's site type information in the dataset
if 'site_type' in ds.coords:
    print("\nSite types:")
    print(np.unique(ds.site_type.values))
elif 'site_type' in ds.data_vars:
    print("\nSite types:")
    print(np.unique(ds.site_type.values))
else:
    print("\nLooking for site type patterns in site names...")
    site_names = ds.site.values
    print("All sites:", site_names)
    
    # Try to infer site types from site names (common patterns)
    site_types = []
    for site in site_names:
        site_str = str(site)
        # Look for common forest type indicators in site names
        if any(indicator in site_str.lower() for indicator in ['dry', 'mesic', 'wet', 'open', 'dense', 'sparse']):
            site_types.append(site_str)
    
    if site_types:
        print("Sites with type indicators:", site_types)

Dataset dimensions and coordinates:
FrozenMappingWarningOnValuesAccess({'plot': 315, 'z': 108})

Dataset coordinates:
Coordinates:
  * z          (z) float64 864B 0.0 1.0 2.0 3.0 4.0 ... 104.0 105.0 106.0 107.0
  * plot       (plot) <U12 15kB 'AGG_O_01_P1' 'AGG_O_01_P2' ... 'ULY_Y_96_P5'
    site       (plot) <U9 11kB 'AGG_O_01' 'AGG_O_01' ... 'ULY_Y_96' 'ULY_Y_96'
    site_type  (plot) <U3 4kB 'AGG' 'AGG' 'AGG' 'AGG' ... 'ULY' 'ULY' 'ULY'

Dataset data variables:
Data variables:
    max                  (plot) float64 3kB 16.28 17.18 21.46 ... 19.32 35.28
    min                  (plot) float64 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    range                (plot) float64 3kB 16.28 17.18 21.46 ... 19.32 35.28
    mean                 (plot) float64 3kB 4.428 4.471 5.346 ... 1.36 1.908
    median               (plot) float64 3kB 4.555 4.654 5.666 ... 0.56 0.376
    sd                   (plot) float64 3kB 4.093 4.085 4.651 ... 2.601 4.168
    var                  (plot) float64 3kB 16.

In [ ]:
# This slice is actually > 1m as i represents (i-1, i] m
z_slice = slice(2, None)

def plot_site(site):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # Plot 1: inside_pct
    selected = ds.where(ds['site'].isin([site]), drop=True)
    selected['inside_pct'].sel(z=z_slice).plot.line(y='z', hue='plot', ax=ax1)

    # Add mean line for inside_pct
    mean_inside = selected['inside_pct'].sel(z=z_slice).mean(dim='plot')
    mean_inside.plot.line(y='z', ax=ax1, color='black', linewidth=2, label='Mean', linestyle='--')

    ax1.set_xlabel('Inside Percentage (%)')
    ax1.set_ylabel('Height (m)')
    ax1.set_title(f'Inside Percentage - Site {site}')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, None)
    ax1.set_ylim(2, None)



    # Plot 2: pad
    selected['pad'].pipe(lambda ds: ds / 0.5).sel(z=z_slice).plot.line(y='z', hue='plot', ax=ax2, alpha=1, linewidth=1)

    # Add mean line for pad
    mean_pad = selected['pad'].pipe(lambda ds: ds / 0.5).sel(z=z_slice).mean(dim='plot')
    mean_pad.plot.line(y='z', ax=ax2, color='black', linewidth=2, label='Mean', linestyle='--')



    ax2.set_xlabel('Plant Area Density (k = 0.5)')
    ax2.set_ylabel('Height (m)')
    ax2.set_title(f'Plant Area Density - Site {site}')
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(0, None)
    ax2.set_ylim(2, None)

    plt.tight_layout()

sites_dropdown = widgets.Dropdown(
    options=sites,
    value=sites[1],
    description='site:',
    disabled=False,
)

interactive_plot = widgets.interactive(plot_site, site=sites_dropdown)
display(interactive_plot)

interactive(children=(Dropdown(description='site:', index=1, options=(np.str_('AGG_O_01'), np.str_('AGG_O_05')…

# Site type level height profiles

In [39]:
site_types = np.unique(ds.site_type.values)

In [ ]:
# This slice is actually > 1m as i represents (i-1, i] m
z_slice = slice(2, None)

def plot_site_type(site_type):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # Filter data for the selected site type
    selected = ds.where(ds['site_type'].isin([site_type]), drop=True)
    
    # Get unique sites within this site type
    sites_in_type = np.unique(selected.site.values)
    
    # Plot 1: inside_pct
    # Plot a line for each site (which is the mean of plots in that site)
    for site in sites_in_type:
        site_data = selected.where(selected['site'].isin([site]), drop=True)
        site_mean_inside = site_data['inside_pct'].sel(z=z_slice).mean(dim='plot')
        site_mean_inside.plot.line(y='z', ax=ax1, linewidth=2, label=f'Site {site}')
    
    # Add overall mean line for all sites in this site type
    overall_mean_inside = selected['inside_pct'].sel(z=z_slice).mean(dim='plot')
    overall_mean_inside.plot.line(y='z', ax=ax1, color='black', linewidth=2, label='Mean of all sites', linestyle='--')

    ax1.set_xlabel('Inside Percentage (%)')
    ax1.set_ylabel('Height (m)')
    ax1.set_title(f'Inside Percentage - Site Type {site_type}')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, None)
    ax1.set_ylim(2, None)
    # Limit legend entries to avoid clutter
    handles, labels = ax1.get_legend_handles_labels()
    if len(handles) > 10:  # If too many sites, only show mean
        ax1.legend([handles[-1]], [labels[-1]], loc='upper right')
    else:
        ax1.legend(loc='upper right')

    # Plot 2: pad
    # Plot a line for each site (which is the mean of plots in that site)
    for site in sites_in_type:
        site_data = selected.where(selected['site'].isin([site]), drop=True)
        site_mean_pad = site_data['pad'].pipe(lambda ds: ds / 0.5).sel(z=z_slice).mean(dim='plot')
        site_mean_pad.plot.line(y='z', ax=ax2, linewidth=2, label=f'Site {site}')
    
    # Add overall mean line for all sites in this site type
    overall_mean_pad = selected['pad'].pipe(lambda ds: ds / 0.5).sel(z=z_slice).mean(dim='plot')
    overall_mean_pad.plot.line(y='z', ax=ax2, color='black', linewidth=2, label='Mean of all sites', linestyle='--')

    ax2.set_xlabel('Plant Area Density (k = 0.5)')
    ax2.set_ylabel('Height (m)')
    ax2.set_title(f'Plant Area Density - Site Type {site_type}')
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(0, None)
    ax2.set_ylim(2, None)
    # Limit legend entries to avoid clutter
    handles, labels = ax2.get_legend_handles_labels()
    if len(handles) > 10:  # If too many sites, only show mean
        ax2.legend([handles[-1]], [labels[-1]], loc='upper right')
    else:
        ax2.legend(loc='upper right')

    plt.tight_layout()

site_types_dropdown = widgets.Dropdown(
    options=site_types,
    value=site_types[1],
    description='Site Type:',
    disabled=False,
)

interactive_plot_site_type = widgets.interactive(plot_site_type, site_type=site_types_dropdown)
display(interactive_plot_site_type)

interactive(children=(Dropdown(description='Site Type:', index=1, options=(np.str_('AGG'), np.str_('EPO'), np.…